In [65]:
import pandas as pd
import numpy as np
import openpyxl
from datetime import datetime
import gc

In [66]:
files = ['WP020204 - 20-Feb-2025 12-22-34-000 PM - MainEnergy.xls', 'WP020204 - 20-Feb-2025 12-22-34-000 PM - TODEnergy.xls']

In [67]:
serial_no = files[0].split("-")[0].strip()
serial_no

'WP020204'

In [68]:
# Read the Excel file
df = pd.read_excel(files[0])
df = df.drop(index=[0, 1]).reset_index(drop=True) #drop the first two rows

# assign values of the third row as column names
df.columns = df.iloc[0]
df = df.drop(0).reset_index(drop=True)
#df.head()

In [69]:
column_list = {"Billing Date":'billing_period',"Active(I) Total":"cumulative_import",
    "Active(E) Total":"cumulative_export","Apparent-Active(I) - type 2":"apparent_import",
    "Reactive(I)":"reactive_import","Reactive(E)":"reactive_export"}

df = df.rename(columns=column_list)
df = df[df["History"] != 0] #drop this row because we can not parse its date string (its empty)
df['billing_period'] = pd.to_datetime(df['billing_period']).dt.normalize()
df['month'] = pd.to_datetime(df['billing_period']).dt.month_name()
df['year'] = pd.to_datetime(df['billing_period']).dt.year

# select needed columns
column_list = ['History', 'billing_period', 'cumulative_import', 'cumulative_export',
      'apparent_import', 'reactive_import', 'reactive_export','month', 'year']
df = df[column_list]
#df = df[df["History"] == 1]
df.head()

,History,billing_period,cumulative_import,cumulative_export,apparent_import,reactive_import,reactive_export,month,year
1,1,2025-02-01,176406172.578899,45036.513,187551818.269299,51925429.9622,899196.0148,February,2025
2,2,2025-01-01,170227521.205,43139.556,180877936.302899,49919915.6973,883925.8009,January,2025
3,3,2024-12-01,164393093.199699,42768.7731,174709372.8919,48128523.3858,882430.1424,December,2024
4,4,2024-11-01,158545581.6502,42182.8183,168517109.958799,46323894.1451,878456.6002,November,2024
5,5,2024-10-01,152804300.599,42182.8055,162411024.3161,44461302.9224,874835.434,October,2024


### GETTING TOU VALUES

In [70]:
df2 = pd.read_excel(files[1])
df2 = df2.drop(index=[0, 1]).reset_index(drop=True) #drop the first two rows

# assign values of the third row as column names
df2.columns = df2.iloc[0]
df2 = df2.drop(0).reset_index(drop=True)
df2_columns = ['History', 'Billing Date', 'TOD/SLAB', 'Active(I) Total','Active(E) Total']
df2 = df2[df2_columns]
df2.head(3)

,History,Billing Date,TOD/SLAB,Active(I) Total,Active(E) Total
0,0,----,1,96256715.1213,25016.595
1,0,----,2,58640935.1716,1532.7879
2,0,----,3,25970470.2811,18548.5367


In [71]:
df2_columns = {'Billing Date':'billing_period', 'TOD/SLAB':'tou', 'Active(I) Total':'import',
               'Active(E) Total':'export'}
df2 = df2.rename(columns=df2_columns)
df2 = df2[df2["History"] != 0] #drop the current values. also, we cannot parse its date(empty)
df2['billing_period'] = pd.to_datetime(df2['billing_period']).dt.normalize()
df2['month'] = pd.to_datetime(df2['billing_period']).dt.month_name()
df2['year'] = pd.to_datetime(df2['billing_period']).dt.year
#df2 = df2[df2["History"] == 1]
df2.head()

,History,billing_period,tou,import,export,month,year
3,1,2025-02-01,1,93887499.2142,24955.1705,February,2025
4,1,2025-02-01,2,57194922.7297,1532.7879,February,2025
5,1,2025-02-01,3,25204181.6104,18548.5367,February,2025
6,2,2025-01-01,1,90692175.6319,24859.7326,January,2025
7,2,2025-01-01,2,55184111.7331,1532.7879,January,2025


In [72]:
history_list = df2['History'].tolist()

In [73]:
# rates1, rates2, rates3, rates4, rates5, rates6 = [],[],[],[],[],[]
# for history in history_list:
#     # getting rates and and assiging them to df
#     df2['rate1'] = df2[(df2['tou'] == 1) & (df2['History'] == history)]['import'].values[0]
#     df2['rate2'] = df2[(df2['tou'] == 2) & (df2['History'] == history)]['import'].values[0]
#     df2['rate3'] = df2[(df2['tou'] == 3) & (df2['History'] == history)]['import'].values[0]

#     df2['rate4'] = df2[(df2['tou'] == 1) & (df2['History'] == history)]['export'].values[0]
#     df2['rate5'] = df2[(df2['tou'] == 2) & (df2['History'] == history)]['export'].values[0]
#     df2['rate6'] = df2[(df2['tou'] == 3) & (df2['History'] == history)]['export'].values[0]

# df2

generate a dictionary 
dict_rates = {1:{'rate1':8494,"rate2":0,"rate3":90}}

In [74]:
# df['rate1'] = df['rate1'].apply(lambda x: '{:.2f}'.format(x))
# df['rate1']

In [75]:
rate1,rate2,rate3,rate4,rate5,rate6 = {},{},{},{},{},{}

# Iterate through unique history values
for hist_value in sorted(df2['History'].unique()):
    # Filter rows where history == hist_value and tou == 3
    rate1_value = df2.loc[(df2['History'] == hist_value) & (df2['tou'] == 1), 'import']
    rate2_value = df2.loc[(df2['History'] == hist_value) & (df2['tou'] == 2), 'import']
    rate3_value = df2.loc[(df2['History'] == hist_value) & (df2['tou'] == 3), 'import']


    rate4_value = df2.loc[(df2['History'] == hist_value) & (df2['tou'] == 1), 'export']
    rate5_value = df2.loc[(df2['History'] == hist_value) & (df2['tou'] == 2), 'export']
    rate6_value = df2.loc[(df2['History'] == hist_value) & (df2['tou'] == 3), 'export']
    
    # Store results in dictionary (or print them)
    if not rate1_value.empty:
        rate1[hist_value] = rate1_value.to_list()[0]

    if not rate2_value.empty:
        rate2[hist_value] = rate2_value.to_list()[0]

    if not rate3_value.empty:
        rate3[hist_value] = rate3_value.to_list()[0]

    if not rate4_value.empty:
        rate4[hist_value] = rate4_value.to_list()[0]

    if not rate5_value.empty:
        rate5[hist_value] = rate5_value.to_list()[0]

    if not rate6_value.empty:
        rate6[hist_value] = rate6_value.to_list()[0]

In [76]:
# Combine into a DataFrame
df_rates = pd.DataFrame([rate1, rate2, rate3, rate4, rate5, rate6]).T

# Rename columns
df_rates.columns = ['rate1', 'rate2', 'rate3', 'rate4', 'rate5', 'rate6']

# Reset index to make it a column
df_rates = df_rates.reset_index().rename(columns={'index': 'History'})

# Print the DataFrame
df_rates

,History,rate1,rate2,rate3,rate4,rate5,rate6
0,1,9.388750e+07,5.719492e+07,2.520418e+07,24955.1705,1532.7879,18548.5367
1,2,9.069218e+07,5.518411e+07,2.423166e+07,24859.7326,1532.7879,16747.0176
2,3,8.765660e+07,5.323430e+07,2.338262e+07,24766.8885,1532.7879,16469.0788
3,4,8.457140e+07,5.139116e+07,2.246346e+07,24758.8072,1532.7722,15891.2210
4,5,8.148861e+07,4.951845e+07,2.167766e+07,24758.8048,1532.7691,15891.2137
5,6,7.830484e+07,4.771855e+07,2.084247e+07,24757.7735,1532.7691,15877.9340
6,7,7.517876e+07,4.575378e+07,1.991538e+07,18296.8922,1532.7691,15720.2225
7,8,7.200004e+07,4.378396e+07,1.893194e+07,18296.8897,1532.7691,15720.2225
8,9,6.846119e+07,4.162404e+07,1.769508e+07,18296.8597,1532.7691,15720.2225
9,10,6.488120e+07,3.947051e+07,1.651096e+07,18296.8595,1532.7691,15720.2207


In [77]:
df_rates['serial_no'] = serial_no 
# df['reading_date'] = datetime.now().replace(second=0, microsecond=0)
# df['vt_ratio'] = pd.NA  
# df['ct_ratio'] = pd.NA  
# df['no_of_resets'] = pd.NA  
# df['date_of_last_reset'] = pd.NA  
# df['programing_count'] = pd.NA 
# df['vt_ratio'] = pd.NA  
# df['ct_ratio'] = pd.NA  
# df['no_of_resets'] = pd.NA  
# df['date_of_last_reset'] = pd.NA  
# df['programing_count'] = pd.NA 
 

In [78]:
columns_to_assign_na = ['vt_ratio', 'ct_ratio', 'no_of_resets', 'date_of_last_reset', 'programing_count',
                        'max_dem1','max_dem1_datetime','max_dem2','max_dem2_datetime','max_dem3','max_dem3_datetime']

for column in columns_to_assign_na:
    df_rates[column] = pd.NA

df_rates

,History,rate1,rate2,rate3,rate4,rate5,rate6,serial_no,vt_ratio,ct_ratio,no_of_resets,date_of_last_reset,programing_count,max_dem1,max_dem1_datetime,max_dem2,max_dem2_datetime,max_dem3,max_dem3_datetime
0,1,9.388750e+07,5.719492e+07,2.520418e+07,24955.1705,1532.7879,18548.5367,WP020204,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2,9.069218e+07,5.518411e+07,2.423166e+07,24859.7326,1532.7879,16747.0176,WP020204,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,3,8.765660e+07,5.323430e+07,2.338262e+07,24766.8885,1532.7879,16469.0788,WP020204,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,4,8.457140e+07,5.139116e+07,2.246346e+07,24758.8072,1532.7722,15891.2210,WP020204,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,5,8.148861e+07,4.951845e+07,2.167766e+07,24758.8048,1532.7691,15891.2137,WP020204,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,6,7.830484e+07,4.771855e+07,2.084247e+07,24757.7735,1532.7691,15877.9340,WP020204,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,7,7.517876e+07,4.575378e+07,1.991538e+07,18296.8922,1532.7691,15720.2225,WP020204,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,8,7.200004e+07,4.378396e+07,1.893194e+07,18296.8897,1532.7691,15720.2225,WP020204,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,9,6.846119e+07,4.162404e+07,1.769508e+07,18296.8597,1532.7691,15720.2225,WP020204,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,10,6.488120e+07,3.947051e+07,1.651096e+07,18296.8595,1532.7691,15720.2207,WP020204,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [79]:
df_final = pd.merge(df, df_rates, on='History', how='inner')
df_final

,History,billing_period,cumulative_import,cumulative_export,apparent_import,reactive_import,reactive_export,month,year,rate1,...,ct_ratio,no_of_resets,date_of_last_reset,programing_count,max_dem1,max_dem1_datetime,max_dem2,max_dem2_datetime,max_dem3,max_dem3_datetime
0,1,2025-02-01,176406172.578899,45036.513,187551818.269299,51925429.9622,899196.0148,February,2025,9.388750e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2,2025-01-01,170227521.205,43139.556,180877936.302899,49919915.6973,883925.8009,January,2025,9.069218e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,3,2024-12-01,164393093.199699,42768.7731,174709372.8919,48128523.3858,882430.1424,December,2024,8.765660e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,4,2024-11-01,158545581.6502,42182.8183,168517109.958799,46323894.1451,878456.6002,November,2024,8.457140e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,5,2024-10-01,152804300.599,42182.8055,162411024.3161,44461302.9224,874835.434,October,2024,8.148861e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,6,2024-09-01,146985425.6646,42168.4945,156193793.7461,42540535.0956,868022.7941,September,2024,7.830484e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,7,2024-08-01,140967484.871299,35549.9017,149830925.7882,40759525.8709,862382.4484,August,2024,7.517876e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,8,2024-07-01,134835499.7527,35549.8992,143413429.5976,39285588.1761,786495.2135,July,2024,7.200004e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,9,2024-06-01,127899872.042899,35549.8692,136387477.194299,38604269.7574,591937.5956,June,2024,6.846119e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,10,2024-05-01,120982242.390099,35549.8672,129367589.8303,37712654.6068,529072.4632,May,2024,6.488120e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [80]:
del df, df2, df_rates
gc.collect()

3301

In [82]:
df_final.to_excel("final_df.xlsx")